In [31]:
#import all neccessary lib
import numpy as np
import cv2
import os
import glob
import random
np.random.seed(0)

### Define variables

In [32]:
# define custom variables
cat = 0
dog = 1
height = 130
width = 130

cat_train_filepath = 'data/cat_dog_ds/train/cats/*'
dog_train_filepath = 'data/cat_dog_ds/train/dogs/*'
cat_test_filepath = 'data/cat_dog_ds/test/cats/*'
dog_test_filepath = 'data/cat_dog_ds/test/dogs/*'

### Define functions

In [33]:
#define function which read train cats and dog dataset and create training dataset with labels
def create_x_y_set(filepath,height,width,label):
    down_width = width
    down_height = height
    down_points = (down_width, down_height)
    pics = glob.glob(filepath)
    train_images = []
    for pic in pics:
        img = cv2.imread(pic, cv2.IMREAD_COLOR)
        resized_down = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)
        cat_img = resized_down.reshape(height*width*3,1)/255
        train_images.append(cat_img)
    train_images = np.array(train_images).reshape(height*width*3,len(pics))
    train_label = np.array([label] * len(pics)).reshape(1,len(pics))
    return train_images, train_label

#create test train dataset
def create_train_test_data(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath):
    cat_X_train, cat_y_train = create_x_y_set(cat_train_filepath,height,width,cat)
    dog_X_train, dog_y_train = create_x_y_set(dog_train_filepath,height,width,dog)
    cat_X_test, cat_y_test = create_x_y_set(cat_test_filepath,height,width,cat)
    dog_X_test, dog_y_test = create_x_y_set(dog_test_filepath,height,width,dog)
    X_train = np.concatenate([cat_X_train,dog_X_train],axis=1)
    y_train = np.concatenate([cat_y_train,dog_y_train],axis=1)
    X_test = np.concatenate([cat_X_test,dog_X_train],axis=1)
    y_test = np.concatenate([cat_y_test,dog_y_train],axis=1)
    return X_train, X_test, y_train, y_test


#define sigmoid function
def sigmoid_fun(x):
    s = 1/(1+ np.exp(-x))
    return s

#define layers parameters
def layer_sizes(X,y):
    '''
    n_x : the size of input layer
    n_y : the side of output layer
    '''
    n_x = X.shape[0]
    n_h = 4
    n_y = y.shape[0]
    return (n_x,n_h,n_y)

In [34]:
X_train, X_test, y_train, y_test = create_train_test_data(cat_train_filepath,dog_train_filepath,cat_test_filepath,dog_test_filepath)
(n_x, n_h, n_y) = layer_sizes(X_train, y_train)

print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

The size of the input layer is: n_x = 50700
The size of the hidden layer is: n_h = 4
The size of the output layer is: n_y = 1


In [35]:
#next step is initialize the models parameter
def initialize_parameters(n_x,n_h,n_y):
    '''
    W1 - weight matrix of shape (n_h,n_x)
    b1 - bias vector of shape (n_h,1)
    W2 - weight matrix pf shape (n_y,n_h)
    b2 - bias vector of shape (n_y,1)
    '''
    W1 = np.random.rand(n_h,n_x) * 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.rand(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters


In [36]:
w_b_parameters = initialize_parameters(n_x,n_h,n_y)
print("W1 = " + str(w_b_parameters["W1"]))
print("b1 = " + str(w_b_parameters["b1"]))
print("W2 = " + str(w_b_parameters["W2"]))
print("b2 = " + str(w_b_parameters["b2"]))

W1 = [[5.48813504e-03 7.15189366e-03 6.02763376e-03 ... 5.28349966e-03
  1.91973944e-03 5.81062909e-03]
 [4.47756414e-03 8.96297944e-03 1.48150407e-03 ... 2.04809474e-03
  7.14350906e-03 7.84704175e-03]
 [2.36189840e-03 3.04578969e-03 3.68959161e-03 ... 7.63533677e-05
  3.51341196e-03 5.60464881e-03]
 [2.73435452e-03 6.06041932e-04 9.60744526e-03 ... 5.00545028e-03
  9.33227913e-03 9.55807633e-03]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[0.00360308 0.00309031 0.00102643 0.00847248]]
b2 = [[0.]]


In [37]:
# define forward propogation
def forward_propagation(X,parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    Z1 = np.dot(W1,X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = sigmoid_fun(Z2)

    cache = {
        "Z1":Z1,
        "A1":A1,
        "Z2":Z2,
        "A2":A2
    }

    return A2, cache


In [38]:
A2, cache = forward_propagation(X_train, w_b_parameters)

# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))

139.27079738190054 1.0 0.016192294901378742 0.5040479852804864


In [78]:
#define compute function
def compute_cost(A2,y):
    # numbers of example
    m = y.shape[1]
    #compute the cross-entropy cost
    logprobs = logprobs = np.multiply(y ,np.log(A2)) + np.multiply((1-y), np.log(1-A2))
    cost = (-1/m) * np.sum(logprobs)
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
    assert(isinstance(cost, float))
    return cost

In [75]:
print("cost = " + str(compute_cost(A2, y_train)))

cost = 0.6931944892773821


In [73]:
def backward_propagation(parameters,cache, X, y):
    # no of examples
    m = X.shape[1]

    # load the parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b1"]

    #load the cache values
    A1 = cache["A1"]
    A2 = cache["A2"]
    Z1 = cache["Z1"]
    Z2 = cache["Z2"]

    # Backward propagation: calculate dW1, db1, dW2, db2.

    dZ2 = y - A2
    dW2 = (1/m) * np.dot(dZ2,A1.T) 
    db2 = (1/m) * np.sum(dZ2,axis=1,keepdims=True)
    dZ1 = np.dot(W2.T,dZ2) * (1 - np.power(A1,2)) 
    dW1 = (1/m) * np.dot(dZ1,X.T) 
    db1 = (1/m) * np.sum(dZ2,axis=1,keepdims=True)

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [42]:
grads = backward_propagation(w_b_parameters, cache, X_train, y_train)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

dW1 = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
db1 = [[-0.00494565]]
dW2 = [[-0.00494565 -0.00494565 -0.00494565 -0.00494565]]
db2 = [[-0.00494565]]


In [43]:
# write function to update parameters
def update_parameters(parameters, grads, learning_rate):
    # Retrieve each parameter from the dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters



In [44]:
w_b_parameters = update_parameters(w_b_parameters, grads,1.2)

print("W1 = " + str(w_b_parameters["W1"]))
print("b1 = " + str(w_b_parameters["b1"]))
print("W2 = " + str(w_b_parameters["W2"]))
print("b2 = " + str(w_b_parameters["b2"]))

W1 = [[5.48813504e-03 7.15189366e-03 6.02763376e-03 ... 5.28349966e-03
  1.91973944e-03 5.81062909e-03]
 [4.47756414e-03 8.96297944e-03 1.48150407e-03 ... 2.04809474e-03
  7.14350906e-03 7.84704175e-03]
 [2.36189840e-03 3.04578969e-03 3.68959161e-03 ... 7.63533677e-05
  3.51341196e-03 5.60464881e-03]
 [2.73435452e-03 6.06041932e-04 9.60744526e-03 ... 5.00545028e-03
  9.33227913e-03 9.55807633e-03]]
b1 = [[0.00593478]
 [0.00593478]
 [0.00593478]
 [0.00593478]]
W2 = [[0.00953786 0.00902509 0.00696122 0.01440726]]
b2 = [[0.00593478]]


In [90]:
# integrate part all above part
def nn_model(X,y, learning_rate, num_iterations = 10000, print_cost=False):
    n_x = layer_sizes(X, y)[0]
    n_h = layer_sizes(X, y)[1]
    n_y = layer_sizes(X, y)[2]

    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]


    for i in range(num_iterations):
        # calculate yhat by computing forward propagation
        A2, cache = forward_propagation(X,parameters)
        A2[A2 == 0.] = 1e-10
        A2[A2 == 1.] = 1. - 1e-10
        #compute the cost
        cost = compute_cost(A2,y)
        # calculate gradient by computing backward propagation
        grads = backward_propagation(parameters,cache, X, y)
        #adjust the weights
        parameters = update_parameters(parameters, grads, learning_rate)
        # If print_cost=True, Print the cost every 1000 iterations
        # print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return parameters


In [91]:
parameters = nn_model(X_train,y_train, 0.1, num_iterations = 10000, print_cost=True)

Cost after iteration 0: 0.693165
Cost after iteration 10: 0.693346
Cost after iteration 20: 0.695259
Cost after iteration 30: 0.715057
Cost after iteration 40: 0.890024
Cost after iteration 50: 1.633824
Cost after iteration 60: 2.807638
Cost after iteration 70: 4.054886
Cost after iteration 80: 5.308781
Cost after iteration 90: 6.563225
Cost after iteration 100: 7.817713
Cost after iteration 110: 9.072205
Cost after iteration 120: 10.326698
Cost after iteration 130: 11.581190
Cost after iteration 140: 12.835687
Cost after iteration 150: 14.090134
Cost after iteration 160: 15.345742
Cost after iteration 170: 16.606401
Cost after iteration 180: 18.054182
Cost after iteration 190: 11.533595
Cost after iteration 200: 11.533595
Cost after iteration 210: 11.533595
Cost after iteration 220: 11.533595
Cost after iteration 230: 11.533595
Cost after iteration 240: 11.533595
Cost after iteration 250: 11.533595
Cost after iteration 260: 11.533595
Cost after iteration 270: 11.533595
Cost after iter

In [92]:
parameters

{'W1': array([[0.00456502, 0.00067165, 0.00186103, ..., 0.0012986 , 0.00249212,
         0.00569293],
        [0.00227345, 0.00492937, 0.00612524, ..., 0.0044963 , 0.0056905 ,
         0.00461356],
        [0.00611061, 0.00220712, 0.00584752, ..., 0.0059317 , 0.00802096,
         0.00741804],
        [0.00682878, 0.00419289, 0.00203032, ..., 0.00218609, 0.00615592,
         0.00383489]]),
 'b1': array([[499.00842021],
        [499.00842021],
        [499.00842021],
        [499.00842021]]),
 'W2': array([[499.01008853, 499.00963243, 499.0128748 , 499.00983592]]),
 'b2': array([[499.00842021]])}

In [93]:
def predict(parameters, X):
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward_propagation(X, parameters)
    predictions = (A2 > 0.5)
    ### END CODE HERE ###
    
    return predictions

In [94]:
yhat = predict(parameters, X_train)

In [95]:
correct_prediction = round(((y_train==yhat).sum()/y_train.shape[1])*100,2)
correct_prediction

49.91